Build a Web Chatbot that acts like you using Gradio & OpenAI

In [1]:
from dotenv import load_dotenv
from openai import OpenAI
from PyPDF2 import PdfReader
import gradio as gr

In [2]:
load_dotenv(override=True)
openai = OpenAI()

In [3]:
reader = PdfReader("../support_docs/professional_cloud_architect_exam_guide_english.pdf")

In [4]:
data = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        data += text

In [5]:
print(data)

Professional Cloud Architect
Certification exam guide
A Google Cloud Certi ed Professional Cloud Architect enables organizations to leverage
Google Cloud technologies. Through an understanding of cloud architecture and Google
technology, this individual designs, develops, and manages robust, secure, scalable, highly
available, and dynamic solutions to drive business objectives. The Cloud Architect should be
pro cient in all aspects of enterprise cloud strategy, solution design, and architectural best
practices. The Cloud Architect should also be experienced in so ware development
methodologies and approaches including multi-tiered distributed applications which span
multicloud or hybrid environments.
Case studies
During the exam for the Cloud Architect Certi cation, some of the questions may refer you to
a case study that describes a  ctitious business and solution concept. These case studies are
intended to provide additional context to help you choose your answers. Review the case
st

In [6]:
name = "Parth"

In [11]:
system_prompt = f"You are acting as Technical Consultant. You are answering questions on {data} from {name}'s college syllabus, \
    particularly questions related to {name}'s future career options, skills, domain and areas to focus. \
    Your responsibility is to help {name} for choosing the syllabus option as faithfully as possible. \
    You are given a pdf of the syllabus to refere the course of action and tell {name} the suitable career options. \
    Be professional and engaging, as if talking to a potential future developer.\
    If you don't know the answer, say so."

system_prompt += f"With this context, please chat with the user, always staying in character as Technical Consultant."

In [12]:
print(system_prompt)

You are acting as Technical Consultant. You are answering questions on Professional Cloud Architect
Certification exam guide
A Google Cloud Certi ed Professional Cloud Architect enables organizations to leverage
Google Cloud technologies. Through an understanding of cloud architecture and Google
technology, this individual designs, develops, and manages robust, secure, scalable, highly
available, and dynamic solutions to drive business objectives. The Cloud Architect should be
pro cient in all aspects of enterprise cloud strategy, solution design, and architectural best
practices. The Cloud Architect should also be experienced in so ware development
methodologies and approaches including multi-tiered distributed applications which span
multicloud or hybrid environments.
Case studies
During the exam for the Cloud Architect Certi cation, some of the questions may refer you to
a case study that describes a  ctitious business and solution concept. These case studies are
intended to provide

In [13]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content":message}]
    response = openai.chat.completions.create(model = "gpt-4o-mini", messages= messages)
    return response.choices[0].message.content


In [14]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Multi LLM Pipeline : How can we acheive this without any Agentic Framework.
- Ask Another LLM to evaluate answers 
- If the answer fails the evaluation, we are going to be able to rerun and make second call to gpt-4o-mini
- Put everything together into Single workflow

In [16]:
# Create a Pydantic model for Evaluation.

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

In [17]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of Technical Consultant for {name} and is referring the syllabus {data} of {name}'s college . \
The Agent has been instructed to be professional and engaging, as if talking to a potential future developer or who came across the same course of syllabus. "

evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [21]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n {history} \n\n"
    user_prompt += f"Here's the latest message from the User: \n\n {message} \n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n {reply} \n\n"
    user_prompt += f"Please evaluate the response, replying with whether it is acceptable and your feedback."

    return user_prompt

In [22]:
# Will use Gemini as a Evaluator

import os
gemini = OpenAI(
    api_key = os.getenv("GEMINI_API_KEY"),
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"

)

In [ ]:
# Evaluate with Structured output, where it will return the Evaluation object
# Note the way to call the api with structured output (.parse)

def evaluate(reply, message, history) -> Evaluation:
    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [24]:
messages = [{"role":"system", "content":system_prompt}] + [{"role": "user", "content": "what the course is about?"}]
response = openai.chat.completions.create(model = "gpt-4o-mini", messages = messages)
reply = response.choices[0].message.content

In [25]:
reply

"The course is centered around preparing individuals for the Google Cloud Certified Professional Cloud Architect certification. This certification enables professionals to design, develop, and manage robust cloud solutions utilizing Google Cloud technologies effectively. \n\nHere are some key aspects of the course:\n\n1. **Cloud Architecture Design:** It covers the principles of designing cloud solutions that meet both business and technical requirements. This includes understanding cost optimization, application design support, integration with external systems, as well as high availability, scalability, and performance considerations.\n\n2. **Infrastructure Management:** The course teaches how to manage and provision solution infrastructures, including configuring network topologies, storage systems, and compute resources. You will also learn about hybrid networking and multicloud environments.\n\n3. **Security and Compliance:** A significant focus is placed on designing solutions wi

In [26]:
evaluate(reply, "what the course is about?", messages)

Evaluation(is_acceptable=True, feedback="The agent's response is of acceptable quality. It accurately describes the course content, covering key aspects such as cloud architecture design, infrastructure management, security, process optimization, implementation management, and reliability assurance. The response is well-organized, provides clear explanations, and aligns with the information provided in the syllabus. Additionally, the response maintains a professional and engaging tone, making it suitable for a technical consultant interacting with a student or potential developer.")

In [28]:
evaluate(reply, "what the course is about?", messages[:1])


Evaluation(is_acceptable=True, feedback='The agent provided a response that accurately describes the course and is helpful to the user. The information given is on point and is derived from the context syllabus.')

In [29]:
# If the evaluator (Gemini model) says that the answer was not acceptable, then need to rerun
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + f"\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [33]:
# The Complete Workflow
def chat(message, history):
    system = system_prompt
    messages = [{"role": "system", "content":system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply = response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    if evaluation.is_acceptable:
        print("PAssed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)
    return reply

In [ ]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


PAssed evaluation - returning reply
PAssed evaluation - returning reply
PAssed evaluation - returning reply
Failed evaluation - retrying
The agent has been playing a role of a Technical Consultant and should not be generating any joke in the conversation. The agent should stick to its role.
